In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler


data = pd.read_csv('data/data.csv')
scaler = MinMaxScaler()
data[['long', 'lat']] = scaler.fit_transform(data[['long', 'lat']])
data

/var/folders/wc/rqgqs2ld2v9_r289z6535cx80000gn/T/ipykernel_30880/126824350.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,level,long,lat
0,0,0.441570,0.661562
1,0,0.268629,0.756519
2,0,0.351682,0.786664
3,0,0.778563,0.864016
4,0,0.521565,0.546740
...,...,...,...
8953,1,0.372988,0.663793
8954,1,0.527866,0.728454
8955,1,0.421333,0.597353
8956,1,0.156966,0.814186


In [15]:
from sklearn.model_selection import train_test_split
import torch

X = data[['long', 'lat']].values
y = data['level'].values

X = X.reshape(X.shape[0], 1, X.shape[1])  # Reshape to (batch_size, seq_length, input_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

In [16]:
import torch.nn as nn

class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Initialize hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        # Forward propagate RNN
        out, _ = self.rnn(x, h0)
        # Pass the output of the last time step
        out = self.fc(out[:, -1, :])
        return out

# Model parameters
input_size = 2  # 'long' and 'lat'
hidden_size = 256
num_layers = 3
num_classes = 4

model = RNNModel(input_size, hidden_size, num_layers, num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 500
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')


Epoch [10/500], Loss: 1.0877959728240967
Epoch [20/500], Loss: 1.0776666402816772
Epoch [30/500], Loss: 1.0690914392471313
Epoch [40/500], Loss: 1.0672435760498047
Epoch [50/500], Loss: 1.067004680633545
Epoch [60/500], Loss: 1.0665837526321411
Epoch [70/500], Loss: 1.0662583112716675
Epoch [80/500], Loss: 1.0657637119293213
Epoch [90/500], Loss: 1.0651253461837769
Epoch [100/500], Loss: 1.0643820762634277
Epoch [110/500], Loss: 1.0637009143829346
Epoch [120/500], Loss: 1.0632944107055664
Epoch [130/500], Loss: 1.063144564628601
Epoch [140/500], Loss: 1.063069224357605
Epoch [150/500], Loss: 1.063010573387146
Epoch [160/500], Loss: 1.0629782676696777
Epoch [170/500], Loss: 1.0629520416259766
Epoch [180/500], Loss: 1.0629254579544067
Epoch [190/500], Loss: 1.0628998279571533
Epoch [200/500], Loss: 1.0628737211227417
Epoch [210/500], Loss: 1.0628448724746704
Epoch [220/500], Loss: 1.062809944152832
Epoch [230/500], Loss: 1.0627635717391968
Epoch [240/500], Loss: 1.0626987218856812
Epoch 

In [17]:
with torch.no_grad():
    correct = 0
    total = X_test.size(0)
    outputs = model(X_test)
    _, predicted = torch.max(outputs.data, 1)
    correct += (predicted == y_test).sum().item()

    print(f'Accuracy of the RNN model on the test set: {100 * correct / total}%')

Accuracy of the RNN model on the test set: 54.129464285714285%
